# CDCS Data Management

This Notebook details the functions and interactions for managing data in a CDCS instance.  Primarily, this means having an account on the database allowing you to log in to create and modify records. 

Notes on CDCS design:

- Each record is assigned to a user and can be assigned to a workspace.
- Any records without a workspace are only accessible to the assigned user.
- A workspace serves as a group of records that can be made accessible to specific users.
- Each CDCS instance has a "Global Public Workspace" that is meant for all users to be able to see.

In [1]:
from pathlib import Path

from cdcs import CDCS

## 1. Class initialization

A CDCS client manager can be initialized by passing it the host url and authenication information.

Parameters

- __username__: (*str, optional*) Username to log in as. If not given, then a promp will ask for it. An empty string '' username will access the client as an anonymous guest.
- __password__: (*str, optional*) The password associated with the username. If not given and username is not '', then a promp will ask for the password.
- __certification__: (*str, optional*) The path to a certification file, if needed.

In [2]:
curator = CDCS('https://potentials.nist.gov/', username='lmh1')

Enter password for lmh1 @ https://potentials.nist.gov:········


## 2. Query data

The query() method will return *all* matching records that you have access to.

Parameters

- __template__: (*list, str, pandas.Series or pandas.DataFrame, optional*) One or more templates or template titles to limit the search by.
- __title__: (*str, optional*) Record title to limit the search by.
- __keyword__: (*str or list, optional*) Keyword(s) to use for a string-based search of record content.  Only records containing all keywords will be returned. 
- __mongoquery__: (*str or dict, optional*) Mongodb find query to use in limiting searches by record element fields.  Note: only record parsing is supported, not field projection.
- __page__: (*int or None, optional*) If an int, then will return results only for that page of 10 records.  If None (default), then results for all pages will be compiled and returned.

Returns
- (*pandas.DataFrame*) All records matching the search request

Specify a template in the database to interact with

In [3]:
# Note: template should the name of a template in the database you are accessing!
template = 'FAQ'

Use query to fetch records

In [4]:
records = curator.query(template=template)
records

,id,last_modification_date,template,title,user_id,workspace,xml_content,template_title
0,5df2ad2290985100269d7bf5,2019-12-17T19:58:26.591000Z,5df2ad2290985100269d7bf1,lammps,2,5de9613a75d7d40014ffb6fc,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",FAQ
1,5df2ad2290985100349d7c6f,2019-12-17T19:58:26.850000Z,5df2ad2290985100269d7bf1,faq,2,5de9613a75d7d40014ffb6fc,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",FAQ
2,5df2ad2390985100269d7bf9,2019-12-17T19:58:27.099000Z,5df2ad2290985100269d7bf1,submit,2,5de9613a75d7d40014ffb6fc,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",FAQ
3,5df2ad2390985100349d7c73,2019-12-17T19:58:27.329000Z,5df2ad2290985100269d7bf1,ref,2,5de9613a75d7d40014ffb6fc,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",FAQ
4,5df2ad2490985100349d7c77,2019-12-17T19:58:27.558000Z,5df2ad2290985100269d7bf1,manuscript,2,5de9613a75d7d40014ffb6fc,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",FAQ
5,5df2ad2490985100269d7bfd,2019-12-17T19:58:27.787000Z,5df2ad2290985100269d7bf1,formats,2,5de9613a75d7d40014ffb6fc,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",FAQ
6,5df2ad2490985100269d7c01,2019-12-17T19:58:28.042000Z,5df2ad2290985100269d7bf1,graphs,2,5de9613a75d7d40014ffb6fc,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",FAQ


Pick the first record, and see its xml contents

In [5]:
record = records.iloc[0]
content = record.xml_content
print(content)

<faq  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" ><question>Where can I download the LAMMPS molecular dynamics software package?</question><answer><![CDATA[<a href="http://lammps.sandia.gov" class="external">LAMMPS</a> (Large-scale Atomic/Molecular Massively Parallel Simulator) is developed and maintained at  <a href="http://www.sandia.gov" class="external">Sandia National Laboratories</a>.]]></answer></faq>


## 3. Manage records

The methods associated with managing records requires having an account with the corresponding database and the correct privileges.

### 3.1. Upload a new record

New records can be uploaded to the database using the upload_record() method.

Parameters

- **template** (*str or pandas.Series*) The template or template title to associate with the record.
- **filename** (*str, optional*) Name of an XML file whose contents are to be uploaded.  Either filename or content required.
- **content** (*str or bytes, optional*) String content to upload. Either filename or content required.
- **title** (*str, optional*) Title to save the record as.  Optional if filename is given (title will be taken as filename without ext).
- **workspace** (*str or pandas.Series, optional*) If given, the record will be assigned to this workspace after successfully being uploaded.
- **duplicatecheck** (*bool, optional*) If True (default), then a ValueError will be raised if a record already exists with the same template and title.  If False, no check is performed possibly allowing for multiple records with the same title to exist in the database.  Note: this check only searches the records that you have access to, so duplicates are possible from other users.
- **verbose** (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

Use the content from the record queried above to upload a "test" record.

In [6]:
title = 'testrecord1'
curator.upload_record(template=template, title=title, content=content, verbose=True)

record testrecord1 (5f58e3c85b91e5002d87aeba) successfully uploaded.


Alternatively, the names of any local XML files can be specified and the method will automatically read the contents.  The file name without path and extension will be used for the title if no title is given.

Also, note that the workspace is being set to 'Global Public Workspace'.

In [7]:
# Save content to local file
filename = 'testrecord2.xml'
with open(filename, 'w') as f:
    f.write(content)
    
# Upload from file
curator.upload_record(template=template, filename=filename, verbose=True,
                      workspace='Global Public Workspace')

# Delete local file (keep working directory clean)
Path(filename).unlink()

record testrecord2 (5f58e3cf5b91e5003487aebb) successfully uploaded.
record 5f58e3cf5b91e5003487aebb assigned to workspace 5de9613a75d7d40014ffb6fc


### 3.2. Retrieve data records

The records that you own can be accessed using the get_records() and get_record() methods. get_records() will fetch all matching records, while get_record() will fetch a single record if exactly one match is found and will throw an error otherwise.

Parameters

- **template** (*str or pandas.Series, optional*) The template or template title to limit the search by.
- **title** (*str, optional*) The data record title to limit the search by.

In [8]:
curator.get_records(template=template)

,id,last_modification_date,template,title,user_id,workspace,xml_content
0,5df2ad2290985100269d7bf5,2019-12-17T19:58:26.591000Z,5df2ad2290985100269d7bf1,lammps,2,5de9613a75d7d40014ffb6fc,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch..."
1,5df2ad2290985100349d7c6f,2019-12-17T19:58:26.850000Z,5df2ad2290985100269d7bf1,faq,2,5de9613a75d7d40014ffb6fc,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch..."
2,5df2ad2390985100269d7bf9,2019-12-17T19:58:27.099000Z,5df2ad2290985100269d7bf1,submit,2,5de9613a75d7d40014ffb6fc,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch..."
3,5df2ad2390985100349d7c73,2019-12-17T19:58:27.329000Z,5df2ad2290985100269d7bf1,ref,2,5de9613a75d7d40014ffb6fc,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch..."
4,5df2ad2490985100349d7c77,2019-12-17T19:58:27.558000Z,5df2ad2290985100269d7bf1,manuscript,2,5de9613a75d7d40014ffb6fc,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch..."
5,5df2ad2490985100269d7bfd,2019-12-17T19:58:27.787000Z,5df2ad2290985100269d7bf1,formats,2,5de9613a75d7d40014ffb6fc,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch..."
6,5df2ad2490985100269d7c01,2019-12-17T19:58:28.042000Z,5df2ad2290985100269d7bf1,graphs,2,5de9613a75d7d40014ffb6fc,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch..."
7,5f58e3c85b91e5002d87aeba,2020-09-09T14:16:39.869000Z,5df2ad2290985100269d7bf1,testrecord1,2,None,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch..."
8,5f58e3cf5b91e5003487aebb,2020-09-09T14:16:47.859000Z,5df2ad2290985100269d7bf1,testrecord2,2,5de9613a75d7d40014ffb6fc,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch..."


In [9]:
record = curator.get_record(template=template, title='testrecord2')
print(record)

id                                                 5f58e3cf5b91e5003487aebb
last_modification_date                          2020-09-09T14:16:47.859000Z
template                                           5df2ad2290985100269d7bf1
title                                                           testrecord2
user_id                                                                   2
workspace                                          5de9613a75d7d40014ffb6fc
xml_content               <faq  xmlns:xsi="http://www.w3.org/2001/XMLSch...
Name: 0, dtype: object


In [10]:
print(record.xml_content)

<faq  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" ><question>Where can I download the LAMMPS molecular dynamics software package?</question><answer><![CDATA[<a href="http://lammps.sandia.gov" class="external">LAMMPS</a> (Large-scale Atomic/Molecular Massively Parallel Simulator) is developed and maintained at  <a href="http://www.sandia.gov" class="external">Sandia National Laboratories</a>.]]></answer></faq>


### 3.3. Update an existing record

The content of a record in the database can be changed using the update_record() method.  Note that this only changes the record's content and all other metadata (user, workspace, database id,...) will remain unchanged.

- **record** (*pandas.Series, optional*) A previously identified record to delete.  As this uniquely defines a record, the template and title parameters are ignored if given. Can contain the new conte
- **template** (*str or pandas.Series, optional*) The template or template title associated with the record.  template + title values must uniquely identify one record.
- **title** (*str, optional*) Title of the record to delete.  template + title values must uniquely identify one record.
- **filename** (*str or Path, optional*) Path to file containing the new record content to upload. Either filename or content required.
- **content** (*str or bytes, optional*) New content to upload. Either filename or content required.
- **workspace** (*str or pandas.Series, optional*) If given, the record will be assigned to this workspace after successfully being updated.
- **verbose** (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

Change the content of testrecord2 by replacing "download" with "find"

In [11]:
new_content = record.xml_content.replace('download', 'find')
print(new_content)

<faq  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" ><question>Where can I find the LAMMPS molecular dynamics software package?</question><answer><![CDATA[<a href="http://lammps.sandia.gov" class="external">LAMMPS</a> (Large-scale Atomic/Molecular Massively Parallel Simulator) is developed and maintained at  <a href="http://www.sandia.gov" class="external">Sandia National Laboratories</a>.]]></answer></faq>


In [12]:
curator.update_record(record=record, content=new_content, verbose=True)

record testrecord2 (5f58e3cf5b91e5003487aebb) has been updated.


Retrieve the record again from the database showing nothing has changed except modification date and content

In [13]:
record = curator.get_record(template=template, title='testrecord2')
print(record)

id                                                 5f58e3cf5b91e5003487aebb
last_modification_date                          2020-09-09T14:17:10.465000Z
template                                           5df2ad2290985100269d7bf1
title                                                           testrecord2
user_id                                                                   2
workspace                                          5de9613a75d7d40014ffb6fc
xml_content               <faq  xmlns:xsi="http://www.w3.org/2001/XMLSch...
Name: 0, dtype: object


In [14]:
print(record.xml_content)

<faq  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" ><question>Where can I find the LAMMPS molecular dynamics software package?</question><answer><![CDATA[<a href="http://lammps.sandia.gov" class="external">LAMMPS</a> (Large-scale Atomic/Molecular Massively Parallel Simulator) is developed and maintained at  <a href="http://www.sandia.gov" class="external">Sandia National Laboratories</a>.]]></answer></faq>


### 3.4. Assign records to a workspace

The upload_record() and update_record() methods give the option to assign each record to a workspace if wanted.  After a record is added, the workspace can also be changed using the assign_records() method.

Parameters

- __workspace__ (*str or pandas.Series*) The workspace or workspace title to assign the records to.
- __records__ (*pandas.Series or pandas.DataFrame, optional*) Pre-selected records to assign to the workspace.  Cannot be given with ids, template, or title.
- __ids__ (*str or list, optional*) The ID(s) of the records to assign to the workspace.  Selecting records using ids has the least overhead. Cannot be given with records, template, or title.
- __template__ (*str or pandas.Series, optional*) The template or template title of records to assign to the workspace.  Cannot be given with records or ids.
- __title__ (*str, optional*) The title of a record to assign to the workspace. Cannot be given with records or ids.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

In [16]:
curator.assign_records(workspace='Global Public Workspace', title='testrecord1', template=template, verbose=True)

record 5f58e3c85b91e5002d87aeba assigned to workspace 5de9613a75d7d40014ffb6fc


### 3.5. Delete a record

A record can also be deleted by the assigned user using the delete_record() method.

Parameters

- __record__ (*pandas.Series, optional*) A previously identified record to delete.  As this uniquely defines a record, the other parameters are ignored.
- __template__ (*str or pandas.Series, optional*) The template or template title associated with the record.  template + title values must uniquely identify one record.
- __title__ (*str, optional*) Title of the record to delete.  template + title values must uniquely identify one record.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

In [17]:
# Delete by identifying with title + template
curator.delete_record(title='testrecord1', template=template, verbose=True)

record testrecord1 (5f58e3c85b91e5002d87aeba) has been deleted.


In [18]:
# Get a record first
record = curator.get_record(title='testrecord2', template=template)

# Delete by passing Series
curator.delete_record(record, verbose=True)

record testrecord2 (5f58e3cf5b91e5003487aebb) has been deleted.


## 4. Manage blobs (raw files)

The database can also store any non-record files as blob files that can later be retrieved and downloaded all at once.

Create blob file for testing

In [27]:
filename = 'test_blob.txt'

with open(filename, 'w') as f:
    f.write('This is my blob for testing')

### 4.1. Upload blob

Blobs can be added to the database using the upload_blob() method.  When a record is uploaded, it will be associated with the given filename (minus the path).

Parameters

- __filename__ (*str or Path*) The path/name of the file to upload.
- __blobbytes__ (*bytesIO, optional*) Pre-loaded file contents.  Allows for the contents of open file-like objects to be passed in and associated with the filename.  If not given, then the file filename will be read instead.
- __workspace__ (*str or pandas.Series, optional*) If given, the blob will be assigned to this workspace after successfully being uploaded.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

Returns

- __handle__ (*str*) The URL that can be used to retrieve the blob's contents.

In [28]:
handle = curator.upload_blob(filename=filename, verbose=True)
print(handle)

File "test_blob.txt" uploaded as blob "test_blob.txt" (5f58e5085b91e5003487aec3)
https://potentials.nist.gov/rest/blob/download/5f58e5085b91e5003487aec3/


### 4.2 Find blob metadata

The list of all blobs currently in the database, or all with an associated filename can be explored with the get_blobs() method.

Parameters

- __filename__ (*str, optional*) The name of the file to limit the search by.

A single blob can also be retrieved using the get_blob() method based on a filename, if it is unique in the database, or a database id.

Parameters

- __id__ (*str, optional*) The unique ID associated with the blob. Cannot be combined with filename
- __filename__ (*str, optional*) The name of the file to limit the search by, which must be unique.  Cannot be combined with id.

__NOTE__: Exploring blob metadata appears to behave differently for some versions of CDCS, with the filename searches only showing blobs that the user owns, while id searches shows any blobs available to the user, either by being owned or in an available workspace.

- Identifying by filename only works for user blobs
- Identifying by id works for all blobs available to you (user + allowed workspaces)

In [39]:
curator.get_blobs()

,filename,handle,id,upload_date,user_id
0,test_blob.txt,https://potentials.nist.gov/rest/blob/download...,5f58e5085b91e5003487aec3,2020-09-09 14:22:00+00:00,2


In [40]:
blobdata = curator.get_blob(filename=filename)
print(blobdata)

filename                                           test_blob.txt
handle         https://potentials.nist.gov/rest/blob/download...
id                                      5f58e5085b91e5003487aec3
upload_date                            2020-09-09 14:22:00+00:00
user_id                                                        2
Name: 0, dtype: object


In [41]:
curator.get_blob(id=blobdata.id)

id                                      5f58e5085b91e5003487aec3
user_id                                                        2
filename                                           test_blob.txt
handle         https://potentials.nist.gov/rest/blob/download...
upload_date                            2020-09-09 14:22:00+00:00
dtype: object

### 4.3 Retrieve blob contents

The contents of a blob can then be retrieved using the get_blob_contents() method.

Paramters

- __blob__ (*pandas.Series, optional*) The blob metadata for a blob, as recieved from get_blobs() or get_blob().  Cannot be combined with id or filename.
- __id__ (*str, optional*) The unique ID associated with the blob. Cannot be combined with blob or filename.
- __filename__ (*str, optional*) The name of the file to limit the search by, which must be unique.  Cannot be combined with blob or id.

Returns

- __content__ (*bytes*) The blob's bytes contents.

In [33]:
print(curator.get_blob_contents(filename=filename))

b'This is my blob for testing'


Alternatively, the blob can be saved directly to a local file based on its associated filename with the download_blob() method.

Parameters

- __blob__ (*pandas.Series, optional*) The blob metadata for a blob, as recieved from get_blobs() or get_blob().  Cannot be combined with id or filename.
- __id__ (*str, optional*) The unique ID associated with the blob. Cannot be combined with blob or filename.
- __filename__ (*str, optional*) The name of the file to limit the search by, which must be unique.  Cannot be combined with blob or id.
- __savedir__ (*str or Path, optional*) The directory to save the file to.  Default value uses the current working directory.

In [42]:
curator.download_blob(filename=filename)

### 4.4. Assign blobs to a workspace

Just like records, blobs can also be assigned to workspaces.  This can be done during uploading, or it can be done after uploading using the assign_blobs() method.

- __workspace__ (*str or pandas.Series*) The workspace or workspace title to assign the blobs to.
- __blobs__ (*pandas.Series or pandas.DataFrame, optional*) Pre-selected blobs to assign to the workspace.  Cannot be given with ids or filename.
- __ids__ (*str or list, optional*) The ID(s) of the blobs to assign to the workspace.  Selecting blobs using ids has the least overhead. Cannot be given with blobs or filename.
- __filename__ (*str, optional*) The name of the blob file to assign to the workspace.  Cannot be given with blobs or ids.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

In [43]:
curator.assign_blobs(workspace='Global Public Workspace', filename=filename, verbose=True)

blob 5f58e5085b91e5003487aec3 assigned to workspace 5de9613a75d7d40014ffb6fc


### 4.5. Delete blob

Finally, blobs can also be deleted using the delete_blob() method.

- __blob__ (*pandas.Series, optional*) The blob metadata for a blob, as recieved from get_blobs() or get_blob().  Cannot be combined with id or filename.
- __id__ (*str, optional*) The unique ID associated with the blob. Cannot be combined with blob or filename.
- __filename__ (*str, optional*) The name of the file to limit the search by, which must be unique.  Cannot be combined with blob or id.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

In [44]:
curator.delete_blob(filename=filename, verbose=True)

Successfully deleted blob "test_blob.txt" (5f58e5085b91e5003487aec3)


## 5. Supporting methods

### 5.1. Finding workspace metadata

Metadata for the available workspaces can be explored using the get_workspaces() and get_workspace() methods.

Parameters

title: (str, optional) The workspace title to limit the search by.

In [45]:
curator.get_workspaces()

,id,is_public,owner,title
0,5de9613a75d7d40014ffb6fc,True,None,Global Public Workspace


In [46]:
curator.get_workspace(title='Global Public Workspace')

id           5de9613a75d7d40014ffb6fc
is_public                        True
owner                            None
title         Global Public Workspace
Name: 0, dtype: object

The attribute global_workspace also retrieves the Global Public Workspace 

In [47]:
workspace = curator.global_workspace
print(workspace)

id           5de9613a75d7d40014ffb6fc
is_public                        True
owner                            None
title         Global Public Workspace
Name: 0, dtype: object


### 5.2. Finding template metadata

A list of all available templates can be retrieved using the template_titles attribute.

In [48]:
curator.template_titles

['Potential',
 'Family',
 'FAQ',
 'Action',
 'PotentialProperties',
 'Request',
 'free_surface',
 'stacking_fault',
 'dislocation',
 'point_defect',
 'crystal_prototype',
 'potential_LAMMPS',
 'Citation',
 'relaxed_crystal',
 'calculation_diatom_scan',
 'calculation_isolated_atom']

Metadata for the available templates can be explored using the get_templates() and get_template() methods.

Parameters

- __title__ (*str, optional*): The template title to limit the search by.
- __is_disabled__ (*bool, optional*): If True, then disabled templates will be returned.  If False (default), then active templates will be returned.
- __current__ (bool, optional*): If True (default), only current template versions will be returned.
- __useronly__ (*bool, optional*): If True, only a user's templates are returned. If False (default), then all global templates are returned.

In [49]:
curator.get_templates()

,content,dependencies,filename,hash,id,title
0,"<?xml version=""1.0"" encoding=""UTF-8"" standalon...",[],record-interatomic-potential.xsd,68e70e482f446ac79de1ffdc05b769e3d8cc7df4,5df2ac9890985100349d7ad7,Potential
1,"<?xml version=""1.0"" encoding=""UTF-8"" standalon...",[],record-interatomic-potential-family.xsd,493be99cbc23614f898354553ea0d307009673d3,5df2ad2090985100349d7c6b,Family
2,"<?xml version=""1.0"" encoding=""UTF-8"" standalon...",[],record-interatomic-potential-faq.xsd,2dc0be555da041f782a7435c377ea7a20f28d3cd,5df2ad2290985100269d7bf1,FAQ
3,"<?xml version=""1.0"" encoding=""UTF-8"" standalon...",[],record-interatomic-potential-action.xsd,f1c3de8a64553d81bd10cfc0cb24dd3cc129c710,5df2ad2690985100349d7c79,Action
4,"<?xml version=""1.0"" encoding=""UTF-8"" standalon...",[],record-per-potential-properties.xsd,ea7eeed57e0cea9b15dbda4b32c08266201a7e5a,5df2ad6f90985100349d7d57,PotentialProperties
5,"<xsd:schema xmlns:xsd=""http://www.w3.org/2001/...",[],Request3,d108bacceb44a2789544ade71018b7f5ceacca1b,5df2ae1190985100269d7db0,Request
6,"<?xml version=""1.0""?>\r\n<xsd:schema xmlns:xsd...",[],record-free-surface.xsd,8d2ff690d7e5558b851769f75bad8d90af47102d,5f4d442a5b91e5003525dcc4,free_surface
7,"<?xml version=""1.0""?>\n<xsd:schema xmlns:xsd=""...",[],record-stacking-fault.xsd,aaa696ec7723f68e5be8280c8cafab69e08cf3f3,5f4d51375b91e5003425dda1,stacking_fault
8,"<?xml version=""1.0""?>\n<xsd:schema xmlns:xsd=""...",[],record-dislocation.xsd,a7463cde6ce130157c5989528260744f9b580fab,5f4d514e5b91e5002725dd3c,dislocation
9,"<?xml version=""1.0""?>\n<xsd:schema xmlns:xsd=""...",[],record-point-defect.xsd,4c8bc7e9d8369cdea5c61b27b418c84d3a350a2d,5df95659ab2f7c0033824ae2,point_defect


In [50]:
curator.get_template('Citation')

content         <?xml version="1.0" encoding="UTF-8" standalon...
dependencies                                                   []
filename                record-interatomic-potential-citation.xsd
hash                     893099854c8ac392bad133ce71ac34f60407ab6f
id                                       5e0a34edab2f7c00263cf2c6
title                                                    Citation
Name: 0, dtype: object

The get_template_managers() method fetches metadata associated with the various versions of templates in the database. 

Parameters

- __title__ (*str, optional*) The template title to limit the search by.
- __is_disabled__ (*bool, optional*) If True, then disabled templates will be returned.  If False (default), then active templates will bereturned.
- __useronly__ (*bool, optional*) If True, only a user's templates are returned. If False (default), then all global templates arereturned.

In [52]:
curator.get_template_managers('Citation')

,_cls,current,disabled_versions,id,is_disabled,title,user,versions
0,VersionManager.TemplateVersionManager,5e0a34edab2f7c00263cf2c6,[5e0a27c3ab2f7c002d3cf2c7],5e0a27c3ab2f7c002d3cf2c8,False,Citation,None,"[5e0a27c3ab2f7c002d3cf2c7, 5e0a34edab2f7c00263..."
